# Наследование в Python

In [1]:
class Pet:
    def __init__(self, name=None):
        self.name = name

In [2]:
# Наслідування:
class Dog(Pet):
    def __init__(self, name, breed=None):
        super().__init__(name)
        # ↑ викликає ко з батьківського класу
        self.breed = breed
        
    def say(self):
        return "{0}: waw".format(self.name)


dog = Dog('Шарік', 'Доберман')

In [3]:
dog.name

'Шарік'

In [4]:
dog.say

<bound method Dog.say of <__main__.Dog object at 0x10516f438>>

In [5]:
dog.say()

'Шарік: waw'

In [6]:
import json

class ExportJSON:
    def to_json(self):
        return json.dumps({
            'name': self.name,
            'breed': self.breed
        })
    
class ExDog(Dog, ExportJSON):
    pass

dog = ExDog('Belka', breed='Дворняжка')

In [7]:
dog.to_json()

'{"name": "Belka", "breed": "\\u0414\\u0432\\u043e\\u0440\\u043d\\u044f\\u0436\\u043a\\u0430"}'

### Любий клас є нащадком object

In [10]:
print("""int: \t\t{}
Dog: \t\t{}
Dog(Pet): \t{}
Dog <- int: \t{}
    """.format(issubclass(int, object),
              issubclass(Dog, object),
              issubclass(Dog, Pet),
              issubclass(Dog, int)))

int: 		True
Dog: 		True
Dog(Pet): 	True
Dog <- int: 	False
    


In [16]:
%lsmagic

Available line magics:
%alias  %alias_magic  %autoawait  %autocall  %automagic  %autosave  %bookmark  %cat  %cd  %clear  %colors  %config  %connect_info  %cp  %debug  %dhist  %dirs  %doctest_mode  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %lf  %lk  %ll  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %lx  %macro  %magic  %man  %matplotlib  %mkdir  %more  %mv  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %popd  %pprint  %precision  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %rep  %rerun  %reset  %reset_selective  %rm  %rmdir  %run  %save  %sc  %set_env  %store  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%debug  %%file  %%html  %%javascript  %%js  %%latex  %%markdown  %%perl  %%prun  %%pypy  %%python  %%pyth

```mermaid
graph TD;
    Object-->Pet;
    Object-->C(ExportJSON);
    Pet-->Dog;
    Dog-->ExDog;
    C(ExportJSON)-->ExDog;
```
![mermaid diagram](https://mermaidjs.github.io/mermaid-live-editor/#/view/eyJjb2RlIjoiZ3JhcGggVEQ7XG4gICAgT2JqZWN0LS0-UGV0O1xuICAgIE9iamVjdC0tPkMoRXhwb3J0SlNPTik7XG4gICAgUGV0LS0-RG9nO1xuICAgIERvZy0tPkV4RG9nO1xuICAgIEMoRXhwb3J0SlNPTiktLT5FeERvZzsiLCJtZXJtYWlkIjp7InRoZW1lIjoiZGVmYXVsdCJ9fQ)